In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [4]:
import pandas as pd
import json
from datetime import datetime, timedelta

# Load the JSON data from the file
# file_path = 'snow_volume_5_min'
# file_path = 'AAPL_Volume_Footprint.txt'
# file_path = 'MMM_Volume_Footprint.txt'
file_path = 'META_Volume_Footprint.txt'


with open(file_path, 'r') as file:
    file_content = file.read()

# Strip everything after the ~m~98~m~ marker
# marker = '~m~98~m~'
# marker = '~m~152~m~'

marker = '~m~99~m~'
file_content = file_content.split(marker)[0]

# file_content

main_data = json.loads(file_content)

# data_section = main_data['p'][1]['st1']['ns']['d']
data_section = main_data['p'][1]['st16']['ns']['d']

nested_data = json.loads(data_section)
footprint_levels = nested_data['graphicsCmds']['create']['footprintLevels']
df = pd.DataFrame(footprint_levels[0]['data'])

footprints = nested_data['graphicsCmds']['create']['footprints']

df1 = pd.DataFrame(footprints[0]['data'])


In [5]:
df1

,id,index,poc,val,vah,levels
0,2,0,577.915,577.83,578.34,"[{'buyVolume': 0.0, 'sellVolume': 213.0, 'imba..."
1,9,1,577.235,576.98,577.66,"[{'buyVolume': 0.0, 'sellVolume': 694.0, 'imba..."
2,16,2,576.555,575.96,576.98,"[{'buyVolume': 733.0, 'sellVolume': 817.0, 'im..."
3,21,3,576.895,576.47,577.15,"[{'buyVolume': 550.0, 'sellVolume': 1620.0, 'i..."
4,27,4,577.575,577.32,577.83,"[{'buyVolume': 1433.0, 'sellVolume': 2579.0, '..."
...,...,...,...,...,...,...
2526,7939,2526,592.535,592.45,592.62,"[{'buyVolume': 432.0, 'sellVolume': 189.0, 'im..."
2527,7940,2527,592.025,591.77,592.28,"[{'buyVolume': 769.0, 'sellVolume': 958.0, 'im..."
2528,7941,2528,592.705,592.45,592.79,"[{'buyVolume': 0.0, 'sellVolume': 38.0, 'imbal..."
2529,7942,2529,593.045,592.79,593.47,"[{'buyVolume': 480.0, 'sellVolume': 1746.0, 'i..."


In [6]:
df.head()

,id,startIndex,type,price,extendTo
0,3,0,buy,577.66,1.0
1,4,0,buy,577.83,1.0
2,5,0,buy,578.00,1.0
3,6,0,buy,578.17,1.0
4,7,0,buy,578.34,1.0


In [7]:
df1.head()

,id,index,poc,val,vah,levels
0,2,0,577.915,577.83,578.34,"[{'buyVolume': 0.0, 'sellVolume': 213.0, 'imba..."
1,9,1,577.235,576.98,577.66,"[{'buyVolume': 0.0, 'sellVolume': 694.0, 'imba..."
2,16,2,576.555,575.96,576.98,"[{'buyVolume': 733.0, 'sellVolume': 817.0, 'im..."
3,21,3,576.895,576.47,577.15,"[{'buyVolume': 550.0, 'sellVolume': 1620.0, 'i..."
4,27,4,577.575,577.32,577.83,"[{'buyVolume': 1433.0, 'sellVolume': 2579.0, '..."


In [19]:
# # Define the trading holidays
# trading_holidays = [
#     datetime(2024, 1, 1),   # Monday, January 1 - New Year's Day
#     datetime(2024, 1, 15),  # Monday, January 15 - Martin Luther King Jr. Day
#     datetime(2024, 2, 19),  # Monday, February 19 - Presidents' Day
#     datetime(2024, 3, 29),  # Friday, March 29 - Good Friday
#     datetime(2024, 5, 27),  # Monday, May 27 - Memorial Day
#     datetime(2024, 6, 19),  # Wednesday, June 19 - Juneteenth National Independence Day
#     datetime(2024, 7, 4),   # Thursday, July 4 - Independence Day
#     datetime(2024, 9, 2),   # Monday, September 2 - Labor Day
#     datetime(2024, 11, 28), # Thursday, November 28 - Thanksgiving Day
#     datetime(2024, 12, 25), # Wednesday, December 25 - Christmas Day
#     datetime(2025, 1, 1),   # Wednesday, January 1 - New Year's Day
#     datetime(2025, 1, 20),  # Monday, January 20 - Martin Luther King Jr. Day
#     datetime(2025, 2, 17),  # Monday, February 17 - Presidents' Day
#     datetime(2025, 4, 18),  # Friday, April 18 - Good Friday
#     datetime(2025, 5, 26),  # Monday, May 26 - Memorial Day
#     datetime(2025, 6, 19),  # Thursday, June 19 - Juneteenth National Independence Day
#     datetime(2025, 7, 4),   # Friday, July 4 - Independence Day
#     datetime(2025, 9, 1),   # Monday, September 1 - Labor Day
#     datetime(2025, 11, 27), # Thursday, November 27 - Thanksgiving Day
#     datetime(2025, 12, 25)  # Thursday, December 25 - Christmas Day
# ]

# # Initialize the base timestamp for index 2245
# base_timestamp = datetime.strptime('2024-12-02 15:55:00', '%Y-%m-%d %H:%M:%S')
# current_timestamp = base_timestamp

# # Initialize a dictionary to store timestamps for each index
# # index_to_timestamp = {2245: current_timestamp}
# index_to_timestamp = {max(df1['index']): current_timestamp}

# # Market hours
# market_open = timedelta(hours=9, minutes=30)
# market_close = timedelta(hours=15, minutes=55)
# day_increment = timedelta(days=1)
# weekend_days = [5, 6]  # Saturday and Sunday

# # Calculate the timestamps backward in 5-minute intervals, excluding weekends and outside market hours
# # for index in range(2244, -1, -1):
# for index in range(max(df1['index'])-1, -1, -1):
#     # Subtract 5 minutes
#     current_timestamp -= timedelta(minutes=5)
    
#     # Check if current timestamp is before market open
#     while (current_timestamp.time() < (datetime.min + market_open).time() or
#            current_timestamp.time() > (datetime.min + market_close).time() or
#            current_timestamp.weekday() in weekend_days or
#            current_timestamp.date() in [holiday.date() for holiday in trading_holidays]):
#         # Move to previous trading day if before market open
#         if current_timestamp.time() < (datetime.min + market_open).time():
#             current_timestamp = datetime.combine(current_timestamp.date() - day_increment, (datetime.min + market_close).time())
#         else:
#             # Otherwise, just subtract 5 minutes
#             current_timestamp -= timedelta(minutes=5)
        
#         # Skip weekends and trading holidays
#         while current_timestamp.weekday() in weekend_days or current_timestamp.date() in [holiday.date() for holiday in trading_holidays]:
#             current_timestamp -= day_increment
#             current_timestamp = datetime.combine(current_timestamp.date(), (datetime.min + market_close).time())
    
#     # Assign the calculated timestamp to the index
#     index_to_timestamp[index] = current_timestamp

# # Create a list to hold the time series data
# time_series_data = []

# # Iterate over df1 and extract levels data
# for i, row in df1.iterrows():
#     timestamp = index_to_timestamp.get(row['index'])
    
#     if timestamp:
#         levels = row['levels']
#         for level in levels:
#             time_series_data.append({
#                 'timestamp': timestamp,
#                 'price': level['price'],
#                 'buyVolume': level['buyVolume'],
#                 'sellVolume': level['sellVolume'],
#                 'imbalance': level['imbalance'],
#                 'index': row['index']
#             })

# # Create the dataframe from the time series data
# series_df = pd.DataFrame(time_series_data)

# series_df.tail(20)


# Define the trading holidays
trading_holidays = [
    datetime(2024, 1, 1),   # Monday, January 1 - New Year's Day
    datetime(2024, 1, 15),  # Monday, January 15 - Martin Luther King Jr. Day
    datetime(2024, 2, 19),  # Monday, February 19 - Presidents' Day
    datetime(2024, 3, 29),  # Friday, March 29 - Good Friday
    datetime(2024, 5, 27),  # Monday, May 27 - Memorial Day
    datetime(2024, 6, 19),  # Wednesday, June 19 - Juneteenth National Independence Day
    datetime(2024, 7, 4),   # Thursday, July 4 - Independence Day
    datetime(2024, 9, 2),   # Monday, September 2 - Labor Day
    datetime(2024, 11, 28), # Thursday, November 28 - Thanksgiving Day
    datetime(2024, 12, 25), # Wednesday, December 25 - Christmas Day
    datetime(2025, 1, 1),   # Wednesday, January 1 - New Year's Day
    datetime(2025, 1, 20),  # Monday, January 20 - Martin Luther King Jr. Day
    datetime(2025, 2, 17),  # Monday, February 17 - Presidents' Day
    datetime(2025, 4, 18),  # Friday, April 18 - Good Friday
    datetime(2025, 5, 26),  # Monday, May 26 - Memorial Day
    datetime(2025, 6, 19),  # Thursday, June 19 - Juneteenth National Independence Day
    datetime(2025, 7, 4),   # Friday, July 4 - Independence Day
    datetime(2025, 9, 1),   # Monday, September 1 - Labor Day
    datetime(2025, 11, 27), # Thursday, November 27 - Thanksgiving Day
    datetime(2025, 12, 25)  # Thursday, December 25 - Christmas Day
]

# Initialize the base timestamp for index 2245
base_timestamp = datetime.strptime('2024-12-02 15:55:00', '%Y-%m-%d %H:%M:%S')
current_timestamp = base_timestamp

# Initialize a dictionary to store timestamps for each index
# index_to_timestamp = {2245: current_timestamp}
index_to_timestamp = {max(df1['index']): current_timestamp}

# Market hours
market_open = timedelta(hours=9, minutes=30)
market_close = timedelta(hours=15, minutes=55)
special_close = {
    datetime(2024, 11, 29).date(): timedelta(hours=12, minutes=55)  # Special close time on 2024-11-29
}
day_increment = timedelta(days=1)
weekend_days = [5, 6]  # Saturday and Sunday

# Calculate the timestamps backward in 5-minute intervals, excluding weekends and outside market hours
# for index in range(2244, -1, -1):
for index in range(max(df1['index'])-1, -1, -1):
    # Subtract 5 minutes
    current_timestamp -= timedelta(minutes=5)
    
    # Check if current timestamp is before market open
    while (current_timestamp.time() < (datetime.min + market_open).time() or
           current_timestamp.time() > (datetime.min + special_close.get(current_timestamp.date(), market_close)).time() or
           current_timestamp.weekday() in weekend_days or
           current_timestamp.date() in [holiday.date() for holiday in trading_holidays]):
        # Move to previous trading day if before market open
        if current_timestamp.time() < (datetime.min + market_open).time():
            current_timestamp = datetime.combine(current_timestamp.date() - day_increment, (datetime.min + special_close.get(current_timestamp.date() - day_increment, market_close)).time())
        else:
            # Otherwise, just subtract 5 minutes
            current_timestamp -= timedelta(minutes=5)
        
        # Skip weekends and trading holidays
        while current_timestamp.weekday() in weekend_days or current_timestamp.date() in [holiday.date() for holiday in trading_holidays]:
            current_timestamp -= day_increment
            current_timestamp = datetime.combine(current_timestamp.date(), (datetime.min + special_close.get(current_timestamp.date(), market_close)).time())
    
    # Assign the calculated timestamp to the index
    index_to_timestamp[index] = current_timestamp

# Create a list to hold the time series data
time_series_data = []

# Iterate over df1 and extract levels data
for i, row in df1.iterrows():
    timestamp = index_to_timestamp.get(row['index'])
    
    if timestamp:
        levels = row['levels']
        for level in levels:
            time_series_data.append({
                'timestamp': timestamp,
                'price': level['price'],
                'buyVolume': level['buyVolume'],
                'sellVolume': level['sellVolume'],
                'imbalance': level['imbalance'],
                'index': row['index']
            })

# Create the dataframe from the time series data
series_df = pd.DataFrame(time_series_data)

series_df.tail(20)


,timestamp,price,buyVolume,sellVolume,imbalance,index
19859,2024-12-02 15:45:00,592.96,268.0,0.0,,2528
19860,2024-12-02 15:50:00,592.45,480.0,1746.0,sell,2529
19861,2024-12-02 15:50:00,592.62,384.0,3290.0,,2529
19862,2024-12-02 15:50:00,592.79,3089.0,2334.0,,2529
19863,2024-12-02 15:50:00,592.96,6139.0,2994.0,,2529
19864,2024-12-02 15:50:00,593.13,2973.0,2739.0,,2529
19865,2024-12-02 15:50:00,593.30,1783.0,1988.0,,2529
19866,2024-12-02 15:50:00,593.47,1175.0,99.0,,2529
19867,2024-12-02 15:50:00,591.60,0.0,253.0,sell,2529
19868,2024-12-02 15:50:00,591.77,0.0,268.0,,2529


In [20]:
series_df.tail(600)

,timestamp,price,buyVolume,sellVolume,imbalance,index
19279,2024-11-29 12:55:00,573.41,0.0,129.0,,2452
19280,2024-11-29 12:55:00,573.58,1814.0,875.0,buy,2452
19281,2024-11-29 12:55:00,573.75,610.0,915.0,,2452
19282,2024-12-02 09:30:00,577.32,37.0,1432.0,sell,2453
19283,2024-12-02 09:30:00,577.49,0.0,87.0,sell,2453
...,...,...,...,...,...,...
19874,2024-12-02 15:55:00,592.28,12199.0,3260.0,,2530
19875,2024-12-02 15:55:00,592.45,4211.0,3158.0,,2530
19876,2024-12-02 15:55:00,592.62,11663.0,8367.0,buy,2530
19877,2024-12-02 15:55:00,592.79,3106.0,0.0,,2530


In [21]:
series_df.to_csv('meta_5_min_series_data.csv')

In [314]:
# Filter the DataFrame
filtered_df = series_df[series_df['timestamp'] >= '2024-12-02 09:30:00']

In [315]:
filtered_df.reset_index(inplace=True,drop=True)

In [316]:
# Sort by timestamp and price ascending
filtered_df = filtered_df.sort_values(by=['timestamp', 'price']).reset_index(drop=True)


In [317]:
filtered_df['timestamp'].value_counts()

timestamp
2024-12-02 09:30:00    49
2024-12-02 09:55:00    22
2024-12-02 10:10:00    19
2024-12-02 10:00:00    17
2024-12-02 09:35:00    15
                       ..
2024-12-02 14:30:00     4
2024-12-02 13:45:00     3
2024-12-02 14:55:00     3
2024-12-02 13:40:00     3
2024-12-02 14:20:00     3
Name: count, Length: 78, dtype: int64

In [318]:
filtered_df

,timestamp,price,buyVolume,sellVolume,imbalance,index
0,2024-12-02 09:30:00,575.45,29.0,0.0,,2453
1,2024-12-02 09:30:00,575.62,100.0,18.0,buy,2453
2,2024-12-02 09:30:00,575.79,100.0,479.0,both,2453
3,2024-12-02 09:30:00,575.96,103.0,186.0,,2453
4,2024-12-02 09:30:00,576.13,104.0,210.0,,2453
...,...,...,...,...,...,...
592,2024-12-02 15:55:00,592.28,12199.0,3260.0,,2530
593,2024-12-02 15:55:00,592.45,4211.0,3158.0,,2530
594,2024-12-02 15:55:00,592.62,11663.0,8367.0,buy,2530
595,2024-12-02 15:55:00,592.79,3106.0,0.0,,2530


In [319]:
filtered_df[filtered_df['timestamp'] == '2024-12-02 09:30:00']

,timestamp,price,buyVolume,sellVolume,imbalance,index
0,2024-12-02 09:30:00,575.45,29.0,0.0,,2453
1,2024-12-02 09:30:00,575.62,100.0,18.0,buy,2453
2,2024-12-02 09:30:00,575.79,100.0,479.0,both,2453
3,2024-12-02 09:30:00,575.96,103.0,186.0,,2453
4,2024-12-02 09:30:00,576.13,104.0,210.0,,2453
5,2024-12-02 09:30:00,576.30,109.0,1905.0,sell,2453
6,2024-12-02 09:30:00,576.47,299.0,2691.0,sell,2453
7,2024-12-02 09:30:00,576.64,203.0,978.0,sell,2453
8,2024-12-02 09:30:00,576.81,137.0,279.0,,2453
9,2024-12-02 09:30:00,576.98,158.0,48.0,,2453


In [328]:


# Calculate total volume at each price level
filtered_df['totalVolume'] = filtered_df['buyVolume'] + filtered_df['sellVolume']

# Group by timestamp and identify the Point of Control (POC) for each 5-minute interval
def calculate_poc(group):
    poc_price = group.loc[group['totalVolume'].idxmax(), 'price']
    group['poc'] = poc_price
    
    # Calculate highest bid stacked imbalance and ask stacked imbalance
    group['highest_bid_stacked_imbalance'] = group['buyVolume'].max()
    group['highest_ask_stacked_imbalance'] = group['sellVolume'].max()
    
    return group

filtered_df = filtered_df.groupby('timestamp', group_keys=False).apply(calculate_poc)

# Calculate delta (buyVolume - sellVolume)
filtered_df['delta'] = filtered_df['buyVolume'] - filtered_df['sellVolume']

# Calculate total ask imbalance count and highest stacked imbalance count
def calculate_imbalances(group):
    # Total ask imbalance count (where imbalance is 'sell' or 'both')
    ask_imbalance_count = ((group['imbalance'] == 'sell') | (group['imbalance'] == 'both')).sum()
    group['total_ask_imbalance_count'] = ask_imbalance_count

    # Highest stacked ask imbalance count (consecutive 'sell' or 'both' imbalance)
    max_stacked_ask_imbalance = ((group['imbalance'] == 'sell') | (group['imbalance'] == 'both')).astype(int).groupby(((group['imbalance'] != 'sell') & (group['imbalance'] != 'both')).cumsum()).cumsum().max()
    group['highest_stacked_ask_imbalance'] = max_stacked_ask_imbalance

    # Total bid imbalance count (where imbalance is 'buy' or 'both')
    bid_imbalance_count = ((group['imbalance'] == 'buy') | (group['imbalance'] == 'both')).sum()
    group['total_bid_imbalance_count'] = bid_imbalance_count

    # Highest stacked bid imbalance count (consecutive 'buy' or 'both' imbalance)
    max_stacked_bid_imbalance = ((group['imbalance'] == 'buy') | (group['imbalance'] == 'both')).astype(int).groupby(((group['imbalance'] != 'buy') & (group['imbalance'] != 'both')).cumsum()).cumsum().max()
    group['highest_stacked_bid_imbalance'] = max_stacked_bid_imbalance

    return group

filtered_df = filtered_df.groupby('timestamp', group_keys=False).apply(calculate_imbalances)

filtered_df = filtered_df.sort_values(by=['timestamp', 'price']).reset_index(drop=True)

def add_support_resistance_by_timestamp(df):
    # Initialize lists for final results
    support_levels_per_timestamp = []
    resistance_levels_per_timestamp = []

    # Group by timestamp
    grouped = df.groupby('timestamp')

    for timestamp, group in grouped:
        support_levels = []
        resistance_levels = []
        
        # Group consecutive rows with the same imbalance
        for _, sub_group in group.groupby((group['imbalance'] != group['imbalance'].shift()).cumsum()):
            if len(sub_group) >= 3:  # Ensure the sub-group has at least 3 rows
                prices = sub_group['price'].tolist()
                imbalance_type = sub_group['imbalance'].iloc[0]
                
                # Identify support and resistance levels based on imbalance type
                if all(sub_group['imbalance'].isin(['buy', 'both'])):
                    support_levels.append([round(p, 2) for p in prices])
                elif all(sub_group['imbalance'].isin(['sell', 'both'])):
                    resistance_levels.append([round(p, 2) for p in prices])

        # Store levels for the current timestamp
        support_levels_per_timestamp.append((timestamp, support_levels))
        resistance_levels_per_timestamp.append((timestamp, resistance_levels))
    
    # Create new DataFrame columns
    df['support_imbalance'] = df['timestamp'].map(
        dict((timestamp, levels) for timestamp, levels in support_levels_per_timestamp)
    )
    df['resistance_imbalance'] = df['timestamp'].map(
        dict((timestamp, levels) for timestamp, levels in resistance_levels_per_timestamp)
    )

    return df

# Apply the function to the filtered DataFrame
filtered_df = add_support_resistance_by_timestamp(filtered_df)

# Sort by timestamp and price
filtered_df = filtered_df.sort_values(by=['timestamp', 'price']).reset_index(drop=True)

# Group by timestamp and check for unfinished bid and ask auctions
def check_auctions(group):
    min_price_row = group.loc[group['price'].idxmin()]
    max_price_row = group.loc[group['price'].idxmax()]
    
    buy_auction_status = 'incomplete' if min_price_row['buyVolume'] > 0 and min_price_row['sellVolume'] > 0 else 'complete'
    sell_auction_status = 'incomplete' if max_price_row['buyVolume'] > 0 and max_price_row['sellVolume'] > 0 else 'complete'
    
    group['buy_auction_status'] = buy_auction_status
    group['sell_auction_status'] = sell_auction_status
    
    return group

# Apply the auction check function to each group
filtered_df = filtered_df.groupby('timestamp').apply(check_auctions).reset_index(drop=True)

C:\Users\saite\AppData\Local\Temp\ipykernel_27188\1395941622.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = filtered_df.groupby('timestamp', group_keys=False).apply(calculate_poc)
C:\Users\saite\AppData\Local\Temp\ipykernel_27188\1395941622.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = filtered_df.groupby('timestamp', group_keys=False).apply(calculate_imbalances)
C:\

In [329]:
filtered_df

,timestamp,price,buyVolume,sellVolume,imbalance,index,totalVolume,poc,highest_bid_stacked_imbalance,highest_ask_stacked_imbalance,...,total_ask_imbalance_count,highest_stacked_ask_imbalance,total_bid_imbalance_count,highest_stacked_bid_imbalance,support_imbalance,resistance_imbalance,bid_auction,ask_auction,buy_auction_status,sell_auction_status
0,2024-12-02 09:30:00,575.45,29.0,0.0,,2453,29.0,582.93,3413.0,2691.0,...,9,5,21,9,"[[580.55, 580.72, 580.89], [581.23, 581.4, 581...","[[576.3, 576.47, 576.64], [577.15, 577.32, 577...",incomplete,incomplete,complete,incomplete
1,2024-12-02 09:30:00,575.62,100.0,18.0,buy,2453,118.0,582.93,3413.0,2691.0,...,9,5,21,9,"[[580.55, 580.72, 580.89], [581.23, 581.4, 581...","[[576.3, 576.47, 576.64], [577.15, 577.32, 577...",incomplete,incomplete,complete,incomplete
2,2024-12-02 09:30:00,575.79,100.0,479.0,both,2453,579.0,582.93,3413.0,2691.0,...,9,5,21,9,"[[580.55, 580.72, 580.89], [581.23, 581.4, 581...","[[576.3, 576.47, 576.64], [577.15, 577.32, 577...",incomplete,incomplete,complete,incomplete
3,2024-12-02 09:30:00,575.96,103.0,186.0,,2453,289.0,582.93,3413.0,2691.0,...,9,5,21,9,"[[580.55, 580.72, 580.89], [581.23, 581.4, 581...","[[576.3, 576.47, 576.64], [577.15, 577.32, 577...",incomplete,incomplete,complete,incomplete
4,2024-12-02 09:30:00,576.13,104.0,210.0,,2453,314.0,582.93,3413.0,2691.0,...,9,5,21,9,"[[580.55, 580.72, 580.89], [581.23, 581.4, 581...","[[576.3, 576.47, 576.64], [577.15, 577.32, 577...",incomplete,incomplete,complete,incomplete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,2024-12-02 15:55:00,592.28,12199.0,3260.0,,2530,15459.0,592.62,12199.0,8367.0,...,0,0,3,1,[],[],incomplete,complete,complete,complete
593,2024-12-02 15:55:00,592.45,4211.0,3158.0,,2530,7369.0,592.62,12199.0,8367.0,...,0,0,3,1,[],[],incomplete,complete,complete,complete
594,2024-12-02 15:55:00,592.62,11663.0,8367.0,buy,2530,20030.0,592.62,12199.0,8367.0,...,0,0,3,1,[],[],incomplete,complete,complete,complete
595,2024-12-02 15:55:00,592.79,3106.0,0.0,,2530,3106.0,592.62,12199.0,8367.0,...,0,0,3,1,[],[],incomplete,complete,complete,complete


In [ ]:
filtered_df[['timestamp','buy_auction_status','sell_auction_status']].drop_duplicates()

,timestamp,buy_auction_status,sell_auction_status
0,2024-12-02 09:30:00,complete,incomplete
49,2024-12-02 09:35:00,complete,incomplete
64,2024-12-02 09:40:00,complete,incomplete
77,2024-12-02 09:45:00,incomplete,incomplete
89,2024-12-02 09:50:00,complete,incomplete
...,...,...,...
560,2024-12-02 15:35:00,incomplete,incomplete
564,2024-12-02 15:40:00,complete,incomplete
571,2024-12-02 15:45:00,complete,complete
578,2024-12-02 15:50:00,complete,incomplete


In [ ]:
import ast

# Define a function that converts the value to a list if needed
def safe_literal_eval(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except ValueError:
            print(f"Error in evaluating: {val}")
            return val  # Optionally handle bad strings gracefully
    return val

# Apply to the columns
filtered_df['support_imbalance'] = filtered_df['support_imbalance'].apply(safe_literal_eval)
filtered_df['resistance_imbalance'] = filtered_df['resistance_imbalance'].apply(safe_literal_eval)

In [324]:
# Initialize active support and resistance levels
active_support_levels = [item for sublist in filtered_df.loc[0, 'support_imbalance'] for item in sublist]
active_resistance_levels = [item for sublist in filtered_df.loc[0, 'resistance_imbalance'] for item in sublist]

# Function to update active support and resistance levels
def update_active_levels(active_levels, traded_price):
    # Remove levels that have been breached
    return [level for level in active_levels if level != traded_price]

# Group by timestamp and update active levels
active_levels_df = []
for timestamp, group in filtered_df.groupby('timestamp'):
    print(timestamp)
    print(active_support_levels)
    for idx, row in group.iterrows():
        traded_price = round(row['price'],2)
        print(traded_price)
        # Add new support and resistance levels from the current row
        new_support_levels = [item for sublist in row['support_imbalance'] for item in sublist]
        new_resistance_levels = [item for sublist in row['resistance_imbalance'] for item in sublist]
        
        # Update active support and resistance levels with new levels
        active_support_levels = list(set(active_support_levels + new_support_levels))
        active_resistance_levels = list(set(active_resistance_levels + new_resistance_levels))
        
        # Update support levels
        active_support_levels = update_active_levels(active_support_levels, traded_price)
        
        # Update resistance levels
        active_resistance_levels = update_active_levels(active_resistance_levels, traded_price)
    
    # Sort the active support and resistance levels
    active_support_levels = sorted(active_support_levels)
    active_resistance_levels = sorted(active_resistance_levels)
    
    # Append the updated levels to the dataframe
    active_levels_df.append({
        'timestamp': timestamp,
        'active_support_levels': active_support_levels,
        'active_resistance_levels': active_resistance_levels
    })

# Create a DataFrame for active levels
active_levels_df = pd.DataFrame(active_levels_df)

2024-12-02 09:30:00
[580.55, 580.72, 580.89, 581.23, 581.4, 581.57, 581.74, 581.91, 582.08, 582.25, 582.42, 582.59]
575.45
575.62
575.79
575.96
576.13
576.3
576.47
576.64
576.81
576.98
577.15
577.32
577.49
577.66
577.83
578.0
578.17
578.34
578.51
578.68
578.85
579.02
579.19
579.36
579.53
579.7
579.87
580.04
580.21
580.38
580.55
580.72
580.89
581.06
581.23
581.4
581.57
581.74
581.91
582.08
582.25
582.42
582.59
582.76
582.93
583.1
583.27
583.44
583.61
2024-12-02 09:35:00
[580.55, 580.72, 580.89, 581.23, 581.4, 581.57, 581.74, 581.91, 582.08, 582.25, 582.42, 582.59]
582.93
583.1
583.27
583.44
583.61
583.78
583.95
584.12
584.29
584.46
584.63
584.8
584.97
585.14
585.31
2024-12-02 09:40:00
[580.55, 580.72, 580.89, 581.23, 581.4, 581.57, 581.74, 581.91, 582.08, 582.25, 582.42, 582.59]
581.57
581.74
581.91
582.08
582.25
582.42
582.59
582.76
582.93
583.1
583.27
583.44
583.61
2024-12-02 09:45:00
[580.55, 580.72, 580.89, 581.23, 581.4]
581.57
581.74
581.91
582.08
582.25
582.42
582.59
582.76
582.9

In [325]:
temp_df = active_levels_df

temp_df['active_support_levels'] = temp_df['active_support_levels'].apply(str)
temp_df['active_resistance_levels'] = temp_df['active_resistance_levels'].apply(str)
temp_df = temp_df.drop_duplicates().reset_index()

temp_df

,index,timestamp,active_support_levels,active_resistance_levels
0,0,2024-12-02 09:30:00,"[580.55, 580.72, 580.89, 581.23, 581.4, 581.57...","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
1,1,2024-12-02 09:35:00,"[580.55, 580.72, 580.89, 581.23, 581.4, 581.57...","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
2,2,2024-12-02 09:40:00,"[580.55, 580.72, 580.89, 581.23, 581.4]","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
3,3,2024-12-02 09:45:00,"[580.55, 580.72, 580.89, 581.23, 581.4, 582.76...","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
4,4,2024-12-02 09:50:00,"[580.55, 580.72, 580.89, 581.23, 581.4]","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
...,...,...,...,...
73,73,2024-12-02 15:35:00,"[580.55, 580.72, 580.89, 581.23, 581.4, 583.61...","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
74,74,2024-12-02 15:40:00,"[580.55, 580.72, 580.89, 581.23, 581.4, 583.61...","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
75,75,2024-12-02 15:45:00,"[580.55, 580.72, 580.89, 581.23, 581.4, 583.61...","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."
76,76,2024-12-02 15:50:00,"[580.55, 580.72, 580.89, 581.23, 581.4, 583.61...","[576.3, 576.47, 576.64, 577.15, 577.32, 577.49..."


In [326]:
temp_df.loc[77,'active_resistance_levels']

'[576.3, 576.47, 576.64, 577.15, 577.32, 577.49, 577.66, 577.83, 589.56]'

In [271]:
filtered_df[filtered_df['timestamp'] == '2024-12-02 09:50:00']

,timestamp,price,buyVolume,sellVolume,imbalance,index,totalVolume,poc,highest_bid_stacked_imbalance,highest_ask_stacked_imbalance,delta,total_ask_imbalance_count,highest_stacked_ask_imbalance,total_bid_imbalance_count,highest_stacked_bid_imbalance,support_imbalance,resistance_imbalance
89,2024-12-02 09:50:00,582.76,0.0,3.0,,2457,3.0,583.61,1860.0,3118.0,-3.0,1,1,1,1,[],[]
90,2024-12-02 09:50:00,582.93,564.0,904.0,both,2457,1468.0,583.61,1860.0,3118.0,-340.0,1,1,1,1,[],[]
91,2024-12-02 09:50:00,583.10,259.0,3118.0,,2457,3377.0,583.61,1860.0,3118.0,-2859.0,1,1,1,1,[],[]
92,2024-12-02 09:50:00,583.27,1666.0,1947.0,,2457,3613.0,583.61,1860.0,3118.0,-281.0,1,1,1,1,[],[]
93,2024-12-02 09:50:00,583.44,1860.0,1363.0,,2457,3223.0,583.61,1860.0,3118.0,497.0,1,1,1,1,[],[]
94,2024-12-02 09:50:00,583.61,1829.0,2061.0,,2457,3890.0,583.61,1860.0,3118.0,-232.0,1,1,1,1,[],[]
95,2024-12-02 09:50:00,583.78,852.0,571.0,,2457,1423.0,583.61,1860.0,3118.0,281.0,1,1,1,1,[],[]
96,2024-12-02 09:50:00,583.95,763.0,20.0,,2457,783.0,583.61,1860.0,3118.0,743.0,1,1,1,1,[],[]


In [ ]:

# Convert 'support_imbalance' and 'resistance_imbalance' columns to strings
filtered_df['support_imbalance'] = filtered_df['support_imbalance'].apply(str)
filtered_df['resistance_imbalance'] = filtered_df['resistance_imbalance'].apply(str)

# Reset index and filter required columns
temp_filter_df = filtered_df[['timestamp','poc','highest_bid_stacked_imbalance','highest_ask_stacked_imbalance','total_ask_imbalance_count','highest_stacked_ask_imbalance',
                              'total_bid_imbalance_count','highest_stacked_bid_imbalance', 'support_imbalance', 'resistance_imbalance']].drop_duplicates().reset_index(drop=True)

# Create consecutive POC flag and count highest consecutive POC
temp_filter_df['consecutive_poc_flag'] = temp_filter_df['poc'].eq(temp_filter_df['poc'].shift())

# Calculate the highest consecutive POC count
temp_filter_df['highest_consecutive_poc_count'] = temp_filter_df['poc'].groupby((temp_filter_df['poc'] != temp_filter_df['poc'].shift()).cumsum()).transform('count')

In [240]:
temp_filter_df

,timestamp,poc,highest_bid_stacked_imbalance,highest_ask_stacked_imbalance,total_ask_imbalance_count,highest_stacked_ask_imbalance,total_bid_imbalance_count,highest_stacked_bid_imbalance,support_imbalance,resistance_imbalance,consecutive_poc_flag,highest_consecutive_poc_count
0,2024-12-02 09:30:00,582.93,3413.0,2691.0,9,5,21,9,"[[580.55, 580.72, 580.89], [581.23, 581.4, 581...","[[576.3, 576.47, 576.64], [577.15, 577.32, 577...",False,1
1,2024-12-02 09:35:00,584.80,11485.0,4457.0,2,1,3,1,[],[],False,1
2,2024-12-02 09:40:00,583.10,3138.0,2844.0,2,1,2,1,[],[],False,1
3,2024-12-02 09:45:00,581.74,1142.0,1734.0,0,0,6,5,"[[582.76, 582.93, 583.1, 583.27, 583.44]]",[],False,1
4,2024-12-02 09:50:00,583.61,1860.0,3118.0,1,1,1,1,[],[],False,1
...,...,...,...,...,...,...,...,...,...,...,...,...
73,2024-12-02 15:35:00,592.45,2358.0,2572.0,0,0,1,1,[],[],False,1
74,2024-12-02 15:40:00,591.94,2163.0,2158.0,3,2,0,0,[],[],False,1
75,2024-12-02 15:45:00,592.62,2035.0,2985.0,1,1,2,1,[],[],False,1
76,2024-12-02 15:50:00,592.96,6139.0,3290.0,2,1,0,0,[],[],False,1


In [ ]:
filtered_df

In [242]:
filtered_df.loc[0,'resistance_imbalance'] 

'[[576.3, 576.47, 576.64], [577.15, 577.32, 577.49, 577.66, 577.83]]'

In [172]:
temp_filter_df.loc[0,'support_imbalance']

[[580.55, 580.72, 580.89],
 [581.23, 581.4, 581.57, 581.74, 581.91, 582.08, 582.25, 582.42, 582.59],
 [582.76, 582.93, 583.1, 583.27, 583.44],
 [583.61, 583.78, 583.95, 584.12, 584.29, 584.46],
 [584.8, 584.97, 585.14, 585.31, 585.48],
 [585.99, 586.16, 586.33, 586.5, 586.67, 586.84],
 [586.67, 586.84, 587.01, 587.18],
 [588.2, 588.37, 588.54],
 [588.88, 589.05, 589.22],
 [589.73, 589.9, 590.07, 590.24],
 [591.26, 591.43, 591.6],
 [592.28, 592.45, 592.62],
 [593.81, 593.98, 594.15],
 [591.43, 591.6, 591.77, 591.94]]

In [68]:
temp_filter_df[temp_filter_df['consecutive_poc_flag']]

,timestamp,poc,highest_bid_stacked_imbalance,highest_ask_stacked_imbalance,total_ask_imbalance_count,highest_stacked_ask_imbalance,total_bid_imbalance_count,highest_stacked_bid_imbalance,consecutive_poc_flag,highest_consecutive_poc_count
13,2024-12-02 10:35:00,587.35,4426.0,861.0,2,1,1,1,True,2
46,2024-12-02 13:20:00,591.09,4249.0,1262.0,0,0,2,1,True,2
65,2024-12-02 14:55:00,590.92,2789.0,3055.0,1,1,0,0,True,2


In [39]:
# Group by minute to analyze reversal and initiation
def analyze_minute(group):
    bid_prices = group.loc[group['imbalance'] == "sell", "price"]
    ask_prices = group.loc[group['imbalance'] == "buy", "price"]
    close_price = group['price'].iloc[-1]  # Last price in the minute

    # Initialize comment
    comment = ""

    # Check if close price below ask prices (buyers absorbed by sellers)
    if not ask_prices.empty and close_price < ask_prices.min():
        comment = "Buyers absorbed by sellers."

    # Check if close price above bid prices (sellers absorbed by buyers)
    elif not bid_prices.empty and close_price > bid_prices.max():
        comment = "Sellers absorbed by buyers."

    # Fallback comment
    if comment == "":
        comment = "No significant absorption detected."

    return pd.Series({
        "minute": group['timestamp'].iloc[0].minute,  # Add the minute column explicitly
        "close_price": close_price,
        "comment": comment
    })

# Apply analysis for each minute
result_df = filtered_df.groupby(filtered_df['timestamp'].dt.minute).apply(analyze_minute).reset_index(drop=True)

# Ensure 'minute' column in result_df exists
if 'minute' in result_df.columns:
    result_df.rename(columns={"minute": "grouped_minute"}, inplace=True)
else:
    raise ValueError("The 'minute' column was not created in result_df.")

# Add the grouped analysis back to the original DataFrame by merging on the minute
filtered_df['minute'] = filtered_df['timestamp'].dt.minute  # Add minute to the original DataFrame
final_df = filtered_df.merge(result_df, how="left", left_on="minute", right_on="grouped_minute")

# Drop unnecessary columns to clean up
final_df.drop(columns=["minute", "grouped_minute"], inplace=True)

C:\Users\saite\AppData\Local\Temp\ipykernel_32296\1965833506.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['minute'] = filtered_df['timestamp'].dt.minute  # Add minute to the original DataFrame


In [40]:
final_df

,timestamp,price,buyVolume,sellVolume,imbalance,index,close_price,comment
0,2024-11-29 09:30:00,128.24,1032.0,756.0,sell,4606,132.02,Buyers absorbed by sellers.
1,2024-11-29 09:30:00,128.38,0.0,904.0,,4606,132.02,Buyers absorbed by sellers.
2,2024-11-29 09:30:00,128.10,74.0,657.0,,4606,132.02,Buyers absorbed by sellers.
3,2024-11-29 09:30:00,127.96,1219.0,585.0,sell,4606,132.02,Buyers absorbed by sellers.
4,2024-11-29 09:30:00,127.82,1024.0,1495.0,,4606,132.02,Buyers absorbed by sellers.
...,...,...,...,...,...,...,...,...
202,2024-11-29 12:50:00,133.56,1081.0,3640.0,,4646,133.56,Sellers absorbed by buyers.
203,2024-11-29 12:55:00,133.56,11117.0,11835.0,sell,4647,133.28,Buyers absorbed by sellers.
204,2024-11-29 12:55:00,133.70,945.0,661.0,,4647,133.28,Buyers absorbed by sellers.
205,2024-11-29 12:55:00,133.42,3027.0,4472.0,buy,4647,133.28,Buyers absorbed by sellers.


In [41]:
temp_df = final_df[['timestamp','comment']].drop_duplicates()

temp_df

,timestamp,comment
0,2024-11-29 09:30:00,Buyers absorbed by sellers.
7,2024-11-29 09:35:00,Sellers absorbed by buyers.
11,2024-11-29 09:40:00,No significant absorption detected.
15,2024-11-29 09:45:00,Sellers absorbed by buyers.
19,2024-11-29 09:50:00,Sellers absorbed by buyers.
23,2024-11-29 09:55:00,Buyers absorbed by sellers.
26,2024-11-29 10:00:00,Sellers absorbed by buyers.
30,2024-11-29 10:05:00,No significant absorption detected.
35,2024-11-29 10:10:00,No significant absorption detected.
42,2024-11-29 10:15:00,No significant absorption detected.


In [10]:
filtered_df

,timestamp,price,buyVolume,sellVolume,imbalance,index
0,2024-11-29 09:30:00,234.78,0.0,1193.0,,2906
1,2024-11-29 09:30:00,234.84,1145.0,1269.0,,2906
2,2024-11-29 09:30:00,234.90,2290.0,967.0,,2906
3,2024-11-29 09:30:00,234.96,2290.0,646.0,,2906
4,2024-11-29 09:30:00,235.02,382.0,0.0,,2906
...,...,...,...,...,...,...
280,2024-11-29 12:55:00,237.30,5388.0,1265.0,buy,2947
281,2024-11-29 12:55:00,237.36,13753.0,21027.0,buy,2947
282,2024-11-29 12:55:00,237.42,23940.0,27431.0,,2947
283,2024-11-29 12:55:00,237.48,24117.0,11987.0,sell,2947


In [11]:
from sklearn.ensemble import IsolationForest

# Replace missing values in 'imbalance' with a specific value (e.g., 'missing_value')
filtered_df['imbalance'] = filtered_df['imbalance'].fillna('missing_value')

# Encode categorical 'imbalance' feature
filtered_df['imbalance'] = filtered_df['imbalance'].apply(lambda x: 1 if x == 'buy' else -1 if x == 'sell' else 0)

# Convert the timestamp to numerical format (e.g., seconds since epoch)
filtered_df['timestamp'] = filtered_df['timestamp'].astype(np.int64) // 10**9

# Select relevant features for anomaly detection
features = ['timestamp', 'price', 'buyVolume', 'sellVolume', 'imbalance']

# # Initialize and fit the Isolation Forest model
# model = IsolationForest(contamination='auto', random_state=42)
model = IsolationForest(contamination=0.03, random_state=42)
model.fit(filtered_df[features])

# Predict anomalies
filtered_df['anomaly'] = model.predict(filtered_df[features])

C:\Users\saite\AppData\Local\Temp\ipykernel_32296\2088222860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['imbalance'] = filtered_df['imbalance'].fillna('missing_value')
C:\Users\saite\AppData\Local\Temp\ipykernel_32296\2088222860.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['imbalance'] = filtered_df['imbalance'].apply(lambda x: 1 if x == 'buy' else -1 if x == 'sell' else 0)
C:\Users\saite\AppData\Local\Temp\ipykernel_32296\2088222860.py:10: SettingWithCopyWarning: 
A 

In [12]:
filtered_df['anomaly'].value_counts()

anomaly
 1    276
-1      9
Name: count, dtype: int64

In [13]:
# Convert the numerical timestamp back to normal datetime format
filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'], unit='s')

filtered_anomaly = filtered_df[filtered_df['anomaly'] == -1]

C:\Users\saite\AppData\Local\Temp\ipykernel_32296\1014059852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'], unit='s')


In [14]:
filtered_anomaly['timestamp'].value_counts()

timestamp
2024-11-29 12:55:00    3
2024-11-29 09:30:00    2
2024-11-29 10:25:00    1
2024-11-29 11:10:00    1
2024-11-29 11:50:00    1
2024-11-29 12:50:00    1
Name: count, dtype: int64

In [15]:
filtered_anomaly

,timestamp,price,buyVolume,sellVolume,imbalance,index,anomaly
13,2024-11-29 09:30:00,234.12,2746.0,14200.0,0,2906,-1
18,2024-11-29 09:30:00,233.94,0.0,5707.0,-1,2906,-1
103,2024-11-29 10:25:00,235.26,5064.0,11759.0,-1,2917,-1
150,2024-11-29 11:10:00,235.98,54888.0,0.0,1,2926,-1
196,2024-11-29 11:50:00,237.00,74386.0,1535.0,1,2934,-1
264,2024-11-29 12:50:00,237.42,96558.0,1084.0,1,2946,-1
281,2024-11-29 12:55:00,237.36,13753.0,21027.0,1,2947,-1
282,2024-11-29 12:55:00,237.42,23940.0,27431.0,0,2947,-1
283,2024-11-29 12:55:00,237.48,24117.0,11987.0,-1,2947,-1


In [18]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed


# Filter the DataFrame
filtered_df = series_df[(series_df['timestamp'] >= '2024-11-29 09:30:00') & (series_df['timestamp'] < '2024-07-23 15:50:00')]
filtered_df.reset_index(inplace=True,drop=True)

# Replace missing values in 'imbalance' with a specific value (e.g., 'missing_value')
filtered_df['imbalance'] = filtered_df['imbalance'].fillna('missing_value')

# Encode categorical 'imbalance' feature
filtered_df['imbalance'] = filtered_df['imbalance'].apply(lambda x: 1 if x == 'buy' else -1 if x == 'sell' else 0)

# Select relevant features for anomaly detection
features = ['price', 'buyVolume', 'sellVolume', 'imbalance']

# Normalize the data
scaler = MinMaxScaler()
filtered_df[features] = scaler.fit_transform(filtered_df[features])

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required by MinMaxScaler.

In [71]:
# Prepare the data for LSTM (we'll use sequences)
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 10  # Length of the sequences for LSTM
X, y = create_sequences(filtered_df[features].values, SEQ_LENGTH)

# Build LSTM Autoencoder Model
model = Sequential([
    LSTM(128, activation='relu', input_shape=(SEQ_LENGTH, len(features)), return_sequences=True),
    Dropout(0.2),
    LSTM(64, activation='relu', return_sequences=False),
    RepeatVector(SEQ_LENGTH),
    LSTM(64, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(128, activation='relu', return_sequences=True),
    TimeDistributed(Dense(len(features)))
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
history = model.fit(X, X, epochs=50, batch_size=32, validation_split=0.1, shuffle=False)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 10, 128)           68096     
                                                                 
 dropout_2 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 10, 64)           0         
 or)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 10, 64)            33024     
                                                                 
 dropout_3 (Dropout)         (None, 10, 64)            0         
                                                      

In [73]:
# Predict the reconstruction
X_pred = model.predict(X)

# Calculate the reconstruction error
reconstruction_error = np.mean(np.abs(X_pred - X), axis=(1, 2))

# Threshold for anomaly detection
threshold = np.mean(reconstruction_error) + 3 * np.std(reconstruction_error)

# Detect anomalies
anomalies = reconstruction_error > threshold

# Append anomaly scores to the original DataFrame
anomaly_scores = np.zeros(len(filtered_df))
anomaly_scores[SEQ_LENGTH:] = anomalies

filtered_df['anomaly'] = anomaly_scores

14/14 [==============================] - 0s 7ms/step


/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/1529233682.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['anomaly'] = anomaly_scores


In [75]:

# Inverse transform the normalized features back to their original scale
filtered_df[features] = scaler.inverse_transform(filtered_df[features])


/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/984967263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[features] = scaler.inverse_transform(filtered_df[features])


In [76]:
filtered_df[filtered_df['anomaly'] == 1]

,timestamp,price,buyVolume,sellVolume,imbalance,index,anomaly
138,2024-07-23 11:00:00,206.48,8.0,0.0,1.0,2293,1.0
